# Intento 1

In [34]:
#!/usr/bin/python3
import serial
import numpy as np
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import io
import binascii

# from ubi_test import post_request
baud_rate = 115200

#TODO: Esto es temporal. Mejor que el arduino envie un string que este mejor disenado para recuperar los datos.
def obtenerNumero(s):
    l = []
    for t in s.split():
        try:
            l.append(float(t))
        except ValueError:
            pass

    return l

printBytes = False
pieceSize = 7680

if __name__ == '__main__':
    ser = serial.Serial('/dev/ttyACM0', baud_rate, timeout=1)
    ser.reset_input_buffer()
    ser.flush()
    while True:
        try:
            # print("Waiting for reading...")
            if ser.in_waiting > 0:
                print("Got something -------------------------->")
                # Read raw serial port
                data = ser.read(pieceSize) #.rstrip() # just a very high limit of bytes to read
                print(f'data type: {type(data)}')
                print(f'data length: {len(data)}')

                if len(data) == pieceSize:
                    data_pkg = []
                    data_pkg.append(np.array(data))
                    i = 0
                    while True:
                        if ser.in_waiting > 0:
                            i += 1
                            print(f'Got another piece of pkg of len: {len(data)} (#{i})')
                            data = ser.read(pieceSize)
                            data_pkg.append(np.array(data))
                            if len(data) != pieceSize:
                                print(f'Received last piece of size {len(data)}\n')
                                break
                    print(f'Size of final pkg is {len(np.array(data_pkg).flatten())}')
#                     print(f'Last piece was:\n {data_pkg[-1]}')
 
                if len(data) < 10: # only print if data is short
                    print(data)
                    if printBytes:
                        hexBytes = ',0x'.join('{:02x}'.format(x) for x in data)
                        print(f'Data: {hexBytes}\n')
                    else:
                        print(data)

                # # Read decoded serial port
                # line = ser.readline().decode('utf-8').rstrip()

                # #desde 19 hasta 25
                # num = obtenerNumero(line)[0]
                # data = {"histogram_mean": num}
                # print(data)
                # post_request(data)
        except KeyboardInterrupt:
            ser.close()
            break
        

Got something -------------------------->
data type: <class 'bytes'>
data length: 7680
Got another piece of pkg of len: 7680 (#1)
Got another piece of pkg of len: 7680 (#2)
Got another piece of pkg of len: 7680 (#3)
Got another piece of pkg of len: 7680 (#4)
Got another piece of pkg of len: 7680 (#5)
Got another piece of pkg of len: 7680 (#6)
Got another piece of pkg of len: 7680 (#7)
Got another piece of pkg of len: 7680 (#8)
Got another piece of pkg of len: 7680 (#9)
Got another piece of pkg of len: 7680 (#10)
Got another piece of pkg of len: 7680 (#11)
Got another piece of pkg of len: 7680 (#12)
Got another piece of pkg of len: 7680 (#13)
Got another piece of pkg of len: 7680 (#14)
Got another piece of pkg of len: 7680 (#15)
Got another piece of pkg of len: 7680 (#16)
Got another piece of pkg of len: 7680 (#17)
Got another piece of pkg of len: 7680 (#18)
Got another piece of pkg of len: 7680 (#19)
Got another piece of pkg of len: 7680 (#20)
Got another piece of pkg of len: 7680 (#21

In [33]:
len(np.frombuffer(data, dtype=np.uint8))

8216

In [44]:
import cv2

img = np.zeros(320*240*3)
for i in range(len(data_pkg)):
    img[i*7680 : (i+1)*7680] = np.frombuffer(data_pkg[i], dtype=np.uint8)

In [75]:
import time

reshaped_flattened_img = np.zeros(320*240*3)
# [B0, G1, R2, G0, R1, B3, R0, B2, G3, B1, G2, R3, ... B4, G5, R6, G4, R5, B7, R4, B6, G7, B5, G6, R7]

for i in range(0, 320*240*3, 12):
    reshaped_flattened_img[i+0] = img[i+6]
    reshaped_flattened_img[i+1] = img[i+3]
    reshaped_flattened_img[i+2] = img[i+0]
    
    reshaped_flattened_img[i+3] = img[i+4]
    reshaped_flattened_img[i+4] = img[i+1]
    reshaped_flattened_img[i+5] = img[i+9]
    
    reshaped_flattened_img[i+6] = img[i+2]
    reshaped_flattened_img[i+7] = img[i+10]
    reshaped_flattened_img[i+8] = img[i+7]
    
    reshaped_flattened_img[i+9] = img[i+11]
    reshaped_flattened_img[i+10] = img[i+8]
    reshaped_flattened_img[i+11] = img[i+5]


reshaped_img = np.zeros((320,240,3))
# [R0, G0, B0, R1, G1, B1, ...]
for i in range(320):
    for j in range(240):
        reshaped_img[i,j,0] = reshaped_flattened_img[(i*240+j)*3 + 0]
        reshaped_img[i,j,1] = reshaped_flattened_img[(i*240+j)*3 + 1]
        reshaped_img[i,j,2] = reshaped_flattened_img[(i*240+j)*3 + 2]

# Intento 2: Convertir desde JPEG a RGB888

In [9]:
#!/usr/bin/python3
import random
import serial
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import io
import binascii
import numpy as np

# from ubi_test import post_request
baud_rate = 115200

#TODO: Esto es temporal. Mejor que el arduino envie un string que este mejor disenado para recuperar los datos.
def obtenerNumero(s):
    l = []
    for t in s.split():
        try:
            l.append(float(t))
        except ValueError:
            pass

    return l

if __name__ == '__main__':
    ser = serial.Serial('/dev/ttyACM0', baud_rate, timeout=1)
    ser.flush()
    while True:
        try:
            # print("Waiting for reading...")
            if ser.in_waiting > 0:
                print("Got something")
                # Read raw serial port
                data = ser.read(20000)#.rstrip() # 6000 is just a very high limit of bytes to read
                # print(data)

                # r_data = binascii.unhexlify(data)
                r_data = data
                #r_data = "".unhexlify(chr(int(b_data[i:i+2],16)) for i in range(0, len(b_data),2))

                try:
                    stream = io.BytesIO(r_data)

                    img = Image.open(stream)
                    draw = ImageDraw.Draw(img)
                    print(f'Read image of length {len(r_data)}, size {np.array(img).shape} with format {img.format}')
                    img.show()

                    img_name = "a_test" + str(int(random.random()*100)) + ".png"
                    img.save(img_name)
                except Exception as e:
                    print(f'Error reading data of len {len(data)} because of: {e}')

                # print(data)

                # # Read decoded serial port
                # line = ser.readline().decode('utf-8').rstrip()

                # #desde 19 hasta 25
                # num = obtenerNumero(line)[0]
                # data = {"histogram_mean": num}
                # print(data)
                # post_request(data)
        except KeyboardInterrupt:
            ser.close()
            break
        

Got something
Read image of length 3118, size (240, 320, 3) with format JPEG


In [13]:
np.array(img)[0,0,:]

array([ 87, 130,  59], dtype=uint8)

In [12]:
np.array(img.convert('RGB'))[0,0,:]

array([ 87, 130,  59], dtype=uint8)